In [1]:
import numpy as np
import os
import sys
dir_now = os.getcwd()
sys.path.append(dir_now)
sys.path.append(os.path.join(dir_now, '../'))
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
# method = METHOD_MITHRAL
# method = METHOD_PQ
method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE
nbits = 8

In [3]:
linear_name = 'fc1'
feedback_bits = 256
ncodebooks=1024
ncentroids=16
if method == METHOD_MITHRAL:
    ncentroids = 16
train_sam_num = 3000 # 训练集样本数
test_sam_num = 1000 # 测试集样本数
batch_size = 32
if method == METHOD_EXACT:
    ncodebooks = 0
    ncentroids = 0

In [4]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'fc1in_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'fc1y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'fc1out_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'fc1in_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
    featurepath_test = 'fc1out_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
    y_test = 'fc1y_test_f%i_sam%i.npy' % (feedback_bits, test_sam_num)
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'fc1_w_f256.npy'
biaspath = 'fc1_b_f256.npy'
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileNotFoundError:
    os.makedirs(dir_result)
except FileExistsError:
    pass 

In [5]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method], nbits=nbits)

running method:  Exact


/data/hdr/pq/bolt/experiments/python/CsiTransformerAMM/../matmul.py:334: UserWarning: Persisting input arguments took 0.90s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  est = _fitted_est_for_hparams(


In [6]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [7]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, -1, y_out_last.shape[-1])
print("y_out_last_re.shape: ", y_out_last_re.shape)
if method == METHOD_EXACT:
    train_sam_num = 0 # 训练集样本数
if method == METHOD_SCALAR_QUANTIZE:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_nbits%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, nbits)), y_out_last.astype(np.float32))
else:
    np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last)

[[ 0.08966328 -0.2999047   0.21042289 ... -0.432809    0.1166189
  -0.09447896]
 [-0.00607073  0.09805991 -0.6019249  ... -0.26815078 -0.21811649
  -0.16891438]
 [ 0.00285284  0.1910082   0.23010652 ... -0.24622092 -0.01241541
  -0.02052457]
 ...
 [ 0.3494149   0.13614489 -0.52086323 ... -0.16958523 -0.20233363
   0.00491567]
 [ 0.24282715  0.00705452 -0.24619828 ...  0.06533774 -0.00213837
   0.05453311]
 [-0.3146975   0.10873571 -0.42488495 ... -0.33963907 -0.14223702
   0.17434503]]
y_out_last.shape:  (32000, 256)
y_out_last_re.shape:  (1000, 32, 256)
